We will perform dNdS using probabilities for single base substitutions (single nucleotide context)

Do we have patients with 2 mutations in a single gene?

General steps:
1. Get the probabilities for all 6 types of single base substitutions
   - C>A
   - C>G
   - C>T
   - T>A
   - T>C
   - T>G
2. 

In [104]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import poisson
import os

# change working directory to project-2
if os.getcwd().split('/')[-1] != 'project-2':
    os.chdir('../../../')

In [88]:
# set paths
METADATA = 'data/raw/TCGA.BRCA.metadata.txt'
MUTATIONS = 'data/processed/TCGA.BRCA.mutations.qc1.txt'
CDS_LENGTHS = 'data/processed/gencode.v23lift37.pc_transcripts.transcripts_in_TCGA_MAF.cds_lengths.tsv'
dnds_opportunities = 'data/processed/dnds_opportunities.tsv'

df_opportunities = pd.read_csv(dnds_opportunities, sep='\t')
df_mut = pd.read_csv(MUTATIONS, sep='\t')
df_meta = pd.read_csv(METADATA, sep='\t')
df_cds = pd.read_csv(CDS_LENGTHS, sep='\t')

In [89]:
file = "data/processed/gencode.v23lift37.pc_transcripts.transcripts_in_TCGA_MAF.fa"
df = pd.read_csv(file, sep='\t')

In [90]:
df_cds.head()

,Hugo_Symbol,Transcript_ID,CDS_length,CDS sequence
0,A1BG,ENST00000263100,1488,ATGTCCATGCTCGTGGTCTTTCTCTTGCTGTGGGGTGTCACCTGGG...
1,A1CF,ENST00000373995,1785,ATGGAAGCAGTGTGTCTGGGCACATGCCCAGAGCCAGAAGCGAGCA...
2,A2M,ENST00000318602,4425,ATGGGGAAGAACAAACTCCTTCATCCAAGTCTGGTTCTTCTCCTCT...
3,A2ML1,ENST00000299698,4365,ATGTGGGCTCAGCTCCTTCTAGGAATGTTGGCCCTATCACCAGCCA...
4,A4GALT,ENST00000401850,1062,ATGTCCAAGCCCCCCGACCTCCTGCTGCGGCTGCTCCGGGGCGCCC...


In [91]:
import re
from collections import defaultdict
import pandas as pd

# Mutation count table: SBS_count[ref][alt]
SBS_count = defaultdict(lambda: defaultdict(int))

def normalize_sbs(ref, alt):
    # Normalize purine ref bases to pyrimidine (G→C, A→T; and likewise alt)
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    if ref in ['G', 'A']:
        ref = complement[ref]
        alt = complement[alt]
    return ref, alt

for hgvsc in df_mut['HGVSc']:
    m = re.search(r'c\.\d+([ACGT])>([ACGT])', str(hgvsc))
    if not m:
        continue
    ref, alt = m.group(1), m.group(2)
    norm_ref, norm_alt = normalize_sbs(ref, alt)
    SBS_count[norm_ref][norm_alt] += 1

records = []
for ref in ['C', 'T']:
    for alt in ['A', 'G', 'T'] if ref == 'C' else ['A', 'C', 'G']:
        count = SBS_count[ref][alt]
        records.append({'ref': ref, 'alt': alt, 'mutation': f"{ref}>{alt}", 'count': count})

sbs_df = pd.DataFrame(records)

# Count total number of 'C' and 'T' bases across all CDSs
base_counts = defaultdict(int)

for seq in df_cds['CDS sequence']:
    seq = str(seq).upper()
    base_counts['C'] += seq.count('C')
    base_counts['T'] += seq.count('T')

# Multiply by 2 * number of individuals (diploid genomes)
# change num individuals to number of unique patient ids
num_individuals = df_mut['patient_id'].nunique()
multiplier = 2 * num_individuals

# Assign opportunities for each of the 6 SBS types
opportunities = {
    'C>A': base_counts['C'] * multiplier,
    'C>G': base_counts['C'] * multiplier,
    'C>T': base_counts['C'] * multiplier,
    'T>A': base_counts['T'] * multiplier,
    'T>C': base_counts['T'] * multiplier,
    'T>G': base_counts['T'] * multiplier
}

sbs_df['opportunities'] = sbs_df['mutation'].map(opportunities)
sbs_df['rate'] = sbs_df['count'] / sbs_df['opportunities']

In [92]:
from collections import defaultdict
from itertools import product

# DNA codon table
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',

    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',

    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',

    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
}


mutation_rate_map = sbs_df.set_index('mutation')['rate'].to_dict()

codon_opportunity = {}

for codon in codon_table:
    syn = 0.0
    nonsyn = 0.0
    ref_aa = codon_table[codon]

    for pos in range(3):
        ref_base = codon[pos]
        for alt_base in "ACGT":
            if alt_base == ref_base:
                continue
            mut_codon = codon[:pos] + alt_base + codon[pos+1:]
            if mut_codon not in codon_table:
                continue
            alt_aa = codon_table[mut_codon]

            norm_ref, norm_alt = normalize_sbs(ref_base, alt_base)
            key = f"{norm_ref}>{norm_alt}"
            rate = mutation_rate_map.get(key, 0.0)

            if ref_aa == alt_aa:
                syn += rate
            else:
                nonsyn += rate

    codon_opportunity[codon] = (syn, nonsyn)


In [93]:
mutation_rate_map

{'C>A': 5.556281509855622e-07,
 'C>G': 8.576720209071026e-07,
 'C>T': 2.1104554030905737e-06,
 'T>A': 2.564460589489971e-07,
 'T>C': 4.579768196187648e-07,
 'T>G': 1.928323451026378e-07}

In [94]:
counts_df = df_mut.groupby(['Hugo_Symbol', 'mutation_type']).size()
counts_df = counts_df.unstack(fill_value=0)
counts_df.head()

mutation_type,non-synonymous,synonymous
Hugo_Symbol,,
A1BG,2,0
A1CF,2,1
A2M,6,1
A2ML1,7,4
A4GALT,1,0


In [95]:
results = []

for idx, row in df_cds.iterrows():
    gene = row['Hugo_Symbol']
    seq = row['CDS sequence'].upper()

    if gene not in counts_df.index:
        continue  # skip genes without observed mutations

    O_NS = counts_df.loc[gene, 'non-synonymous']
    O_S = counts_df.loc[gene, 'synonymous']

    E_S, E_NS = 0.0, 0.0
    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]
        if codon not in codon_opportunity:
            continue
        syn, nonsyn = codon_opportunity[codon]
        E_S += syn
        E_NS += nonsyn

    E_S = E_S * num_individuals
    E_NS = E_NS * num_individuals

    λ = O_S / E_S if E_S > 0 else None
    expected_NS_scaled = λ * E_NS if λ is not None else None
    dnds = O_NS / expected_NS_scaled if expected_NS_scaled else None

    results.append({
        'Hugo_Symbol': gene,
        'synonymous': O_S,
        'observed_nonsynonymous': O_NS,
        'synonymous_opportunity': E_S,
        'nonsynonymous_opportunity': E_NS,
        'λ': λ,
        'dN/dS': dnds
    })

df_dnds = pd.DataFrame(results)


In [96]:
df_dnds.sort_values(by='dN/dS', ascending=False, inplace=True)
df_dnds.head(30)

,Hugo_Symbol,synonymous,observed_nonsynonymous,synonymous_opportunity,nonsynonymous_opportunity,λ,dN/dS
12536,TP53,2,202,0.625519,1.562584,3.197346,40.431359
9040,PIK3CA,6,282,1.087641,3.695716,5.516526,13.831996
4350,FOXA1,1,19,0.952791,1.943567,1.049548,9.314328
6671,LRP2,1,24,5.711257,17.310103,0.175093,7.918507
943,ATP10B,1,16,2.291844,5.379339,0.436330,6.816730
7574,MYO7B,1,13,4.130311,8.015474,0.242113,6.698797
6891,MAP2K4,1,18,0.480972,1.456823,2.079124,5.942722
8749,PCDHB7,1,12,1.439062,2.923475,0.694897,5.906925
781,ARID1A,1,15,3.662908,9.445207,0.273007,5.817090
964,ATP2B3,1,11,2.358208,4.625922,0.424051,5.607593


In [97]:
sbs_df

,ref,alt,mutation,count,opportunities,rate
0,C,A,C>A,6203,11163941188,5.556282e-07
1,C,G,C>G,9575,11163941188,8.576720e-07
2,C,T,C>T,23561,11163941188,2.110455e-06
3,T,A,T>A,2447,9541967656,2.564461e-07
4,T,C,T>C,4370,9541967656,4.579768e-07
5,T,G,T>G,1840,9541967656,1.928323e-07


In [106]:
def poisson_pval(x):
    """P-value for observed nonsynonymous mutation count of the gene. Under null, expected should be equal to the observed synonymous mutations times the ratio of nonsynonymous to synonymous opportunities."""
    return 1 - poisson.cdf(x['observed_nonsynonymous']-1, x['synonymous']*x['nonsynonymous_opportunity']/x['synonymous_opportunity'])

df_dnds['poisson_pval'] = df_dnds.apply(lambda x: poisson_pval(x), axis=1)

In [128]:
from src.utils.eval  import evalAccuracy, compareRankings

# load the IntOGen ranking TSV
path_intogen = "data/raw/IntOGen-DriverGenes_TCGA_WXS_BRCA.tsv"
df_intogen = pd.read_csv(path_intogen, sep="\t")

# build a dict mapping gene names to their IntOGen relevance.
#RELEVANCE IS SAMPLES%
baseline_ranks = dict(zip(df_intogen["Symbol"], (df_intogen["Samples (%)"] * 0.01)))

df_dn_ds = df_dnds
#dn_ds_ranks = dict(zip(df_dn_ds["Hugo_Symbol"], df_dn_ds["dN/dS"]))

dn_ds_ranks = dict(zip(df_dn_ds["Hugo_Symbol"], 1- df_dn_ds["poisson_pval"].dropna()))

# calculate accuracy metrics
dcg, bpref, accuracy = evalAccuracy(dn_ds_ranks, baseline_ranks)

df_gene_ranks = compareRankings(dn_ds_ranks, baseline_ranks)

In [129]:
dcg

0.44994484712567173

In [130]:
bpref

0.043478260869565216

In [131]:
accuracy

0.04347826086956519